ニューラルネットワークで学習してみるファイル

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42) # 乱数生成シード
cudnn.benchmark = True

# Grab a GPU if there is one
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cpu


Load dataset

In [2]:
fold = "../train_raw_npy/"
xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}sampled_label.npy")
print(xyz.shape)
print(label.shape)


(196072, 3, 500)
(196072,)


train / test spilit

In [3]:
pin = 170000
train_xyz = xyz[0:pin,:,:]
test_xyz = xyz[pin:190000, :, :]
train_label = label[0:pin]
test_label = label[pin:190000]
print(train_xyz.shape)
print(test_xyz.shape)

(170000, 3, 500)
(20000, 3, 500)


In [4]:
print(pd.Series(train_label).value_counts())
print(pd.Series(test_label).value_counts())

7.0    27641
5.0    26056
6.0    25689
8.0    22016
2.0    21185
1.0    20712
4.0    19603
3.0     7098
Name: count, dtype: int64
7.0    3612
1.0    3315
4.0    2766
6.0    2637
2.0    2489
5.0    2419
8.0    1818
3.0     944
Name: count, dtype: int64


In [5]:
# trainをランダムに20000個サンプリングする。
sample_size = 20000
indices = np.random.choice(train_xyz.shape[0], sample_size, replace=False)
train_xyz_random = train_xyz[indices]
train_label_random = train_label[indices]


In [6]:
print(train_xyz_random.shape)

(20000, 3, 500)


# アーキテクチャデザイン  
オックスフォードでの例にあるようにCNNを使う

In [7]:
class ConvBNReLU(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

            kernel_size=3,
            stride=1,
            padding=1,
            bias=True
    ):
        super(ConvBNReLU, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )
    
    def forward(self,x):
        return self.main(x)


In [ ]:
class CNN(nn.Module):
    def __init__(self, output_size=8, in_channels=3, num_filters_init=8):
        super(CNN, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU(in_channels, num_filters_init, 8,2,3), # 500 -> 250 
            ConvBNReLU(num_filters_init, num_filters_init*2, 6,2,2 ), # 250 -> 125 (248)
            ConvBNReLU(num_filters_init*2, num_filters_init*4,7,2,2 ), # 125 -> 62
            ConvBNReLU(num_filters_init*4, num_filters_init*8,6,2,1 ), # 62 -> 30
            ConvBNReLU(num_filters_init*8, num_filters_init*16, 4, 4, 1), # 30 -> 8
            ConvBNReLU(num_filters_init*16, num_filters_init*32, 8,1,0), # 8 -> 1
            nn.Conv1d(num_filters_init*32, output_size, 1,1,0)
        )
    
    def forward(self, x):
        return self.cnn(x).view(x.shape[0], -1) # ここってもしかして(N, 1500)にしていたりするのかな？

懸念事項１  
cnnのカーネルとかのサイズの計算が合わない可能性がある

懸念事項３  
クラスラベルの変更0~7に  
これは関数側で対応することにする。  
学習時や推論時は考えなくていいようにする。(SHL用の関数を作成するということ)  
0~7クラス分類で学習し、1~8クラス分類として表示する。

In [2]:
def create_dataloader(X, y=None, batch_size=1, shuffle=False):
    if shuffle:
        idxs = np.random.permutation(np.arange(len(X)))
    else:
        idxs = np.arange(len(X))
    
    #データセットをバッチサイズごとに分割
    for i in range(0, len(idxs), batch_size):
        idxs_batch = idxs[i:i+batch_size]
        X_batch = X[idxs_batch].astype('f4')

        X_batch = torch.from_numpy(X_batch)
        if y is None:
            yield X_batch
        else:
            y_batch = y[idxs_batch]
            y_batch = torch.from_numpy(y_batch-1)
            yield X_batch, y_batch

In [8]:
def forward_by_batches(cnn, X):
    Y = []
    cnn.eval()
    with torch.no_grad():
        for x in create_dataloader(X, batch_size=1024, shuffle=False):
            x = x.to(device)
            Y.append(cnn(x))
    cnn.train()

    Y = torch.cat(Y) # Yをテンソルに変換
    return Y

In [9]:
def evaluate_model(cnn, X, Y):
    Y_pred = forward_by_batches(cnn, X)
    loss = F.cross_entropy(Y_pred, torch.from_numpy(Y-1.0).type(torch.int64).to(device)).item() # クロスエントロピー損失の計算 仕方なく0~7クラス分類とする

    Y_pred = F.softmax(Y_pred, dim=1) 
    Y_pred = torch.argmax(Y_pred, dim=1)  # 最も高い確率のY_predのラベルを予測ラベルとしてY_predに入れられる。
    Y_pred = Y_pred + 1  # 予測ラベルに1を加えて1~8の範囲に変換する
    Y_pred = Y_pred.cpu().numpy()  # テンソルでGPUにのっているものをcpuに移動して、それをさらにnumpy配列に変換している。
    kappa = metrics.cohen_kappa_score(Y, Y_pred) # 1~8クラス分類

    return {'loss':loss, 'kappa':kappa, 'Y_pred':Y_pred}

懸念事項２  
クラス分類は1~8である。したがって0~7のクラス分類を1~8に移動させてあげる必要がある。

In [ ]:
num_filters_init = 32 
in_channels = 3  
output_size = len(np.unique(Y))  #　これは分類するラベルの数。最終層の特徴量の数
num_epoch = 5  # num of epochs (full loops though the training set)
lr = 3e-4  # learning rate
batch_size = 32  # size of the mini-batch　バッチサイズを32にしている。

cnn = CNN(
    output_size=output_size,
    in_channels=in_channels,
    num_filters_init=num_filters_init
).to(device)
print(cnn) # ここではcnnのネットワーク構造の定義をして、そのネットワークの構造をprintしている。

loss_fn = nn.CrossEntropyLoss() # クロスエントロピー損失を使用してモデルの出力と実際のラベルとの差を計算するためのインスタンス定義
optimizer = optim.Adam(cnn.parameters(), lr=lr) # オプティマイザーを使用し、設定された学習率lrでモデルのパラメータを最適化する。Adamはよく使われるやつでパフォーマンスが高いらしい　
# optim.Adam()ではcnnのパラメータへの直接参照をすることができるnum_filters

In [ ]:
kappa_history_test = []
loss_history_test = []
loss_history_train = []
losses = []
num_epoch = 10
for i in tqdm(range(num_epoch)): # 同じデータセットでnum_epoch回cnnを学習させている。
    dataloader = create_dataloader(X_train, Y_train, batch_size, shuffle=True)
    for x, target in dataloader:
        x, target = x.to(device), target.type(torch.int64).to(device)
        cnn.zero_grad()
        output = cnn(x) # バッチxをネットワークに通し、予測値outputを取得
        loss = loss_fn(output, target) # 予測値と実際のラベルtargetとの間の損失を計算する。ネットワークの出力outputと正解targetとの間の差異を計算しその結果としてスカラー値lossを生成する。
        loss.backward() # 計算されたlossに従ってネットワークのすべてのパラメータに対する勾配が計算される。
        optimizer.step() # オプティマイザが保存している勾配情報を使用してモデルのパラメータを更新　直接cnnに紐づいているのでcnnのパラメータを直接更新している

        # Logging -- track train loss
        losses.append(loss.item()) # ログを残している。lossというのはパラメータのgradientでパラメータを更新する方向や大きさを決めている。

    # --------------------------------------------------------
    #       Evaluate performance at the end of each epoch
    # --------------------------------------------------------

    # Logging -- average train loss in this epoch
    loss_history_train.append(utils.ewm(losses))

    # Logging -- evalutate performance on test set
    results = evaluate_model(cnn, X_test, Y_test)
    loss_history_test.append(results['loss'])
    kappa_history_test.append(results['kappa'])